# Day 20: HOG特徴量とSVM

## Learning Objectives
- HOG特徴量の抽出方法を理解する
- SVMによる物体分類を理解する
- ヒューマン検出システムを実装する

## HOG特徴量の概要

Histogram of Oriented Gradients (HOG)は、物体検出で広く使われる特徴量です。画像の勾配の方向をヒストグラムで表現します。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

def compute_gradients(image):
    """画像の勾配を計算する"""
    # Sobelフィルタ
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    
    # 勾配の計算
    grad_x = ndimage.convolve(image.astype(float), sobel_x)
    grad_y = ndimage.convolve(image.astype(float), sobel_y)
    
    # 勾配の大きさと方向
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    angle = np.arctan2(grad_y, grad_x) * 180 / np.pi  # 度数法に変換
    
    # 角度を0-360度に正規化
    angle = angle % 360
    
    return magnitude, angle

def create_test_image():
    """テスト用の画像を作成"""
    # 100x100の白い画像
    image = np.ones((100, 100)) * 255
    
    # 矩形パターンを作成
    image[30:70, 30:70] = 100
    
    return image.astype(np.uint8)

# テスト画像の作成
test_image = create_test_image()
plt.imshow(test_image, cmap='gray')
plt.title('Test Image')
plt.axis('off')
plt.show()

In [ ]:
# 勾配の計算
magnitude, angle = compute_gradients(test_image)

# 勾配の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.imshow(magnitude, cmap='gray')
ax1.set_title('Gradient Magnitude')
ax1.axis('off')

# 角度のヒートマップ
ax2.imshow(angle, cmap='hsv', vmin=0, vmax=360)
ax2.set_title('Gradient Angle')
ax2.axis('off')

plt.tight_layout()
plt.show()

## HOG特徴量の実装

In [ ]:
def compute_hog_features(image, cell_size=8, bin_size=9):
    """HOG特徴量を計算する"""
    # 1. 画像の勾配を計算
    magnitude, angle = compute_gradients(image)
    
    # 2. セルに分割
    h, w = image.shape
    cells_h = h // cell_size
    cells_w = w // cell_size
    
    # 3. 各セルのヒストグラムを計算
    hog_features = []
    
    for cy in range(cells_h):
        cell_hog = []
        for cx in range(cells_w):
            # セルの領域を切り出し
            cell_mag = magnitude[cy*cell_size:(cy+1)*cell_size, cx*cell_size:(cx+1)*cell_size]
            cell_angle = angle[cy*cell_size:(cy+1)*cell_size, cx*cell_size:(cx+1)*cell_size]
            
            # ヒストグラムの初期化
            hist = np.zeros(bin_size)
            
            # 角度ビンに割り当て
            bin_width = 360 / bin_size
            
            for y in range(cell_size):
                for x in range(cell_size):
                    if cell_mag[y, x] > 0:  # 勾配が0でないピクセルのみ
                        # 角度をビンインデックスに変換
                        bin_idx = int(cell_angle[y, x] / bin_width) % bin_size
                        hist[bin_idx] += cell_mag[y, x]
            
            cell_hog.append(hist)
        
        hog_features.append(cell_hog)
    
    return np.array(hog_features), (cells_h, cells_w)

# HOG特徴量の計算
hog_features, (cells_h, cells_w) = compute_hog_features(test_image)
print(f"HOG特徴量の形状: {hog_features.shape}")
print(f"セル数: {cells_h} x {cells_w}")
print(f"各セルのヒストグラム（最初のセル）: {hog_features[0, 0]}")

In [ ]:
def visualize_hog_features(image, hog_features):
    """HOG特徴量を可視化する"""
    magnitude, angle = compute_gradients(image)
    h, w = image.shape
    cell_size = h // hog_features.shape[0]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    
    # 原始画像
    ax1.imshow(image, cmap='gray')
    ax1.set_title('Original Image')
    ax1.axis('off')
    
    # HOG特徴量の可視化
    ax2.imshow(image, cmap='gray')
    
    # 各セルに向きの矢印を描画
    for cy in range(hog_features.shape[0]):
        for cx in range(hog_features.shape[1]):
            # セルの中心座標
            center_y = cy * cell_size + cell_size // 2
            center_x = cx * cell_size + cell_size // 2
            
            # ヒストグラムから主な向きを決定
            hist = hog_features[cy, cx]
            dominant_bin = np.argmax(hist)
            
            # ビンの角度を計算
            angle = dominant_bin * (360 / len(hist))
            
            # 矢印の長さ（ヒストグラムの値に比例）
            arrow_length = np.sqrt(np.sum(hist)) * 0.5
            
            # 角度をラジアンに変換
            angle_rad = np.deg2rad(angle)
            
            # 矢印の終点座標
            end_x = center_x + arrow_length * np.cos(angle_rad)
            end_y = center_y + arrow_length * np.sin(angle_rad)
            
            # 矢印を描画
            ax2.arrow(center_x, center_y, end_x - center_x, end_y - center_y,
                     head_width=3, head_length=2, fc='red', ec='red')
    
    ax2.set_title('HOG Features')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

# HOG特徴量の可視化
visualize_hog_features(test_image, hog_features)

## SVMによる物体分類

Support Vector Machine (SVM)を使って、HOG特徴量に基づいて物体を分類します。

In [ ]:
def create_training_data(num_positive=50, num_negative=50):
    """訓練データを作成する"""
    X = []  # 特徴量
    y = []  # ラベル（1: 正例, -1: 負例）
    
    # 正例（ヒューマンのような形状）
    for i in range(num_positive):
        # ランダムな位置に人間のような形状を作成
        h, w = 64, 32
        image = np.ones((h, w)) * 50  # 暗い背景
        
        # 身体（長方形）
        body_y = np.random.randint(10, 40)
        image[body_y:body_y+20, 8:w-8] = 200  # 明るい身体
        
        # 頭（円）
        head_y = body_y - 10
        head_x = w // 2
        for y in range(max(0, head_y-8), min(h, head_y+8)):
            for x in range(max(0, head_x-8), min(w, head_x+8)):
                if (x - head_x)**2 + (y - head_y)**2 <= 64:  # 半径8の円
                    image[y, x] = 200
        
        X.append(image)
        y.append(1)
    
    # 負例（ランダムなノイズ）
    for i in range(num_negative):
        h, w = 64, 32
        image = np.random.randint(0, 255, (h, w))
        
        X.append(image)
        y.append(-1)
    
    return np.array(X), np.array(y)

# 訓練データの作成
X_train, y_train = create_training_data()
print(f"訓練データの形状: {X_train.shape}")
print(f"正例の数: {np.sum(y_train == 1)}")
print(f"負例の数: {np.sum(y_train == -1)}")

# 訓練データの表示
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

# 正例を表示
for i in range(5):
    axes[i].imshow(X_train[i], cmap='gray')
    axes[i].set_title('Positive')
    axes[i].axis('off')

# 負例を表示
for i in range(5, 10):
    axes[i].imshow(X_train[i+num_positive], cmap='gray')
    axes[i].set_title('Negative')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
class SimpleSVM:
    """シンプルなSVM実装"""
    
    def __init__(self, learning_rate=0.01, lambda_param=0.01, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        # 重みの初期化
        self.w = np.zeros(n_features)
        self.b = 0
        
        # 勾配降下法
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                # 条件: y_i * (w · x_i + b) >= 1
                condition = y[idx] * (np.dot(x_i, self.w) + self.b) >= 1
                
                if condition:
                    # マージンから外れている点ではない場合
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    # マージンから外れている点の場合
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y[idx]))
                    self.b -= self.lr * y[idx]
    
    def predict(self, X):
        approx = np.dot(X, self.w) + self.b
        return np.sign(approx)

    def decision_function(self, X):
        """分類関数の値を返す（距離）"""
        return np.dot(X, self.w) + self.b

In [ ]:
def extract_hog_features_batch(images):
    """画像バッチのHOG特徴量を抽出"""
    features = []
    for img in images:
        # 画像を64x32にリサイズ（訓練データと同じサイズ）
        from scipy.ndimage import zoom
        if img.shape != (64, 32):
            img_resized = zoom(img, (64/img.shape[0], 32/img.shape[1]), order=1)
        else:
            img_resized = img
        
        # HOG特徴量を抽出（1次元にフラット化）
        hog_feat, _ = compute_hog_features(img_resized)
        features.append(hog_feat.flatten())
    
    return np.array(features)

# 訓練データからHOG特徴量を抽出
X_train_hog = extract_hog_features_batch(X_train)
print(f"HOG特徴量の形状: {X_train_hog.shape}")

# SVMの学習
svm = SimpleSVM(learning_rate=0.01, n_iters=1000)
svm.fit(X_train_hog, y_train)

print("SVMの学習完了")

## ヒューマン検出システムの実装

In [ ]:
def calculate_iou(box1, box2):
    """2つのバウンディングボックスのIOUを計算"""
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2
    
    # 交差領域を計算
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return 0.0
    
    inter_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    area1 = (x1_max - x1_min) * (y1_max - y1_min)
    area2 = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = area1 + area2 - inter_area
    
    return inter_area / union_area if union_area > 0 else 0.0

def sliding_window_hog_detection(image, window_size, step_size, svm, threshold=0):
    """スライディングウィンドウでヒューマン検出"""
    detections = []
    h, w = image.shape
    win_h, win_w = window_size
    
    for y in range(0, h - win_h + 1, step_size):
        for x in range(0, w - win_w + 1, step_size):
            # ウィンドウを切り出し
            window = image[y:y+win_h, x:x+win_w]
            
            # HOG特徴量を抽出
            hog_feat, _ = compute_hog_features(window)
            features = hog_feat.flatten().reshape(1, -1)
            
            # SVMで予測
            decision_value = svm.decision_function(features)[0]
            
            # しきい値を超えていれば検出
            if decision_value > threshold:
                # スコアを保存
                detections.append((x, y, win_w, win_h, decision_value))
    
    return detections

In [ ]:
# テスト画像を作成（ヒューマンを含む）
def create_test_image_with_human():
    """ヒューマンを含むテスト画像"""
    image = np.ones((200, 200)) * 50  # 暗い背景
    
    # ヒューマン1
    body_y1, body_x1 = 50, 50
    image[body_y1:body_y1+40, body_x1:body_x1+20] = 200  # 身体
    # 頭
    head_y1, head_x1 = body_y1 - 15, body_x1 + 5
    for y in range(max(0, head_y1-10), min(200, head_y1+10)):
        for x in range(max(0, head_x1-10), min(200, head_x1+10)):
            if (x - head_x1)**2 + (y - head_y1)**2 <= 100:
                image[y, x] = 200
    
    # ヒューマン2
    body_y2, body_x2 = 120, 120
    image[body_y2:body_y2+40, body_x2:body_x2+20] = 200  # 身体
    # 頭
    head_y2, head_x2 = body_y2 - 15, body_x2 + 5
    for y in range(max(0, head_y2-10), min(200, head_y2+10)):
        for x in range(max(0, head_x2-10), min(200, head_x2+10)):
            if (x - head_x2)**2 + (y - head_y2)**2 <= 100:
                image[y, x] = 200
    
    return image

# テスト画像の作成と表示
test_image_with_human = create_test_image_with_human()
plt.imshow(test_image_with_human, cmap='gray')
plt.title('Test Image with Humans')
plt.axis('off')
plt.show()

In [ ]:
# ヒューマン検出を実行
detections = sliding_window_hog_detection(
    test_image_with_human,
    window_size=(64, 32),
    step_size=16,
    svm=svm,
    threshold=0
)

print(f"検出されたヒューマン数: {len(detections)}")
for i, (x, y, w, h, score) in enumerate(detections):
    print(f"ヒューマン {i+1}: 位置=({x}, {y}), サイズ=({w}, {h}), スコア={score:.2f}")

In [ ]:
def visualize_human_detections(image, detections):
    """ヒューマン検出結果を可視化"""
    plt.figure(figsize=(10, 8))
    plt.imshow(image, cmap='gray')
    
    # 検出枠を描画
    for x, y, w, h, score in detections:
        rect = Rectangle((x, y), w, h, linewidth=2, edgecolor='red', facecolor='none')
        plt.gca().add_patch(rect)
        # スコアを表示
        plt.text(x, y-5, f'{score:.2f}', color='red', fontsize=10)
    
    plt.title('Human Detection Results')
    plt.axis('off')
    plt.show()

# 検出結果の可視化
visualize_human_detections(test_image_with_human, detections)

In [ ]:
# 誤検出の削除（NMSを使用）
def nms_for_human_detections(detections, iou_threshold=0.3):
    """ヒューマン検出に対するNMS"""
    if len(detections) == 0:
        return []
    
    # スコア順にソート
    detections_sorted = sorted(detections, key=lambda x: x[4], reverse=True)
    
    suppressed = []
    used = [False] * len(detections_sorted)
    
    for i in range(len(detections_sorted)):
        if used[i]:
            continue
        
        suppressed.append(detections_sorted[i])
        used[i] = True
        
        # 重なっている検出を抑制
        for j in range(i+1, len(detections_sorted)):
            if not used[j]:
                box1 = (detections_sorted[i][0], detections_sorted[i][1], 
                       detections_sorted[i][2], detections_sorted[i][3])
                box2 = (detections_sorted[j][0], detections_sorted[j][1], 
                       detections_sorted[j][2], detections_sorted[j][3])
                
                if calculate_iou(box1, box2) > iou_threshold:
                    used[j] = True
    
    return suppressed

# NMSを適用
final_detections = nms_for_human_detections(detections)
print(f"NMS後の検出数: {len(final_detections)}")

# 最終結果の可視化
visualize_human_detections(test_image_with_human, final_detections)

## まとめと次のステップ

### 習得したスキル:
- [x] HOG特徴量の抽出方法を理解した
- [x] SVMによる物体分類を実装した
- [x] ヒューマン検出システムを構築した
- [x] スライディングウィンドウとHOGを組み合わせた

### 考察:
1. HOG特徴量は物体の形状を効果的に捉えられる
2. SVMは2値分離に強く、物体検出に適している
3. スライディングウィンドウとの組み合わせで、画像中の物体を検出できる
4. NMSを使うことで重複検出を削除できる

### 次のステップ:
- より複雑な物体検出（顔検出など）
- 深層学習による特徴量抽出（CNN）
- 実-time物体検出の実装